<a href="https://colab.research.google.com/github/rhehgus02/ESAA_OB/blob/main/%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EC%99%84%EB%B2%BD_%EA%B0%80%EC%9D%B4%EB%93%9C(4)_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%B6%84%EC%84%9D(4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 06. 토픽 모델링(Topic Modeling) - 20 뉴스그룹



---



**토픽 모델링(Topic Modeling)** : 문서 집합에 숨어 있는 주제를 찾아내는 것

-> 머신러닝 기반의 토픽 모델링 적용 : 숨겨진 주제를 효과적으로 표현할 수 있는 중심 단어를 함축적으로 추출

* 자주 사용되는 기법은 **LSA(Latent Semantic Analysis)**와 **LDA(Latent Dirichlet Allocation)**

* 사이킷런은 LDA 기반의 토픽 모델링을 **LatentDirichletAllocation** 클래스로 제공

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 모토사이클, 야구, 그래픽스, 윈도우즈, 중동, 기독교, 전자공학, 의학 8개 주제를 추출.
cats = ['rec.motorcycles', 'rec.sport.baseball', 'comp.graphics', 'comp.windows.x',
        'talk.politics.mideast', 'soc.religion.christian', 'sci.electronics', 'sci.med']

# 위에서 cats 변수로 기재된 카테고리만 추출. fetch_20newsgroups()의 categories에 cats 입력
news_df = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'),
                             categories=cats, random_state=0)   # categories 파라미터 통해 필요한 주제만 필터링해 추출

# LDA는 Count기반의 벡터화만 적용합니다.
count_vect = CountVectorizer(max_df=0.95, max_features=1000, min_df=2, stop_words='english',
                             ngram_range=(1,2))      # max_features로 word 피처의 개수 제한
feat_vect = count_vect.fit_transform(news_df.data)
print('CountVectorizer Shape:', feat_vect.shape)

CountVectorizer Shape: (7862, 1000)


=> CountVectorizer 객체 변수인 feat_vect 모두 7862개의 문서가 1000개의 피처로 구성된 행렬 데이터

* 피처 벡터화된 데이터 세트 기반으로 LDA 토픽 모델링 수행 - LatentDirichletAllocation 클래스
  + **n_components**파라미터 이용해 토픽 개수 조정

In [ ]:
lda = LatentDirichletAllocation(n_components=8, random_state=0)
lda.fit(feat_vect)

LatentDirichletAllocation(n_components=8, random_state=0)

* **components_** 속성값 가지게 돼 : 개별 토픽별로 각 word 피처가 얼마나 많이 그 토픽에 할당됐는지에 대한 수치
  + 높은 값일수록 해당 word 피처는 그 토픽의 중심 word가 된다

In [ ]:
print(lda.components_.shape)
lda.components_

(8, 1000)


array([[3.60992018e+01, 1.35626798e+02, 2.15751867e+01, ...,
        3.02911688e+01, 8.66830093e+01, 6.79285199e+01],
       [1.25199920e-01, 1.44401815e+01, 1.25045596e-01, ...,
        1.81506995e+02, 1.25097844e-01, 9.39593286e+01],
       [3.34762663e+02, 1.25176265e-01, 1.46743299e+02, ...,
        1.25105772e-01, 3.63689741e+01, 1.25025218e-01],
       ...,
       [3.60204965e+01, 2.08640688e+01, 4.29606813e+00, ...,
        1.45056650e+01, 8.33854413e+00, 1.55690009e+01],
       [1.25128711e-01, 1.25247756e-01, 1.25005143e-01, ...,
        9.17278769e+01, 1.25177668e-01, 3.74575887e+01],
       [5.49258690e+01, 4.47009532e+00, 9.88524814e+00, ...,
        4.87048440e+01, 1.25034678e-01, 1.25074632e-01]])

=> 8개의 토픽별로 1000개의 word 피처가 해당 토픽별로 연관도 값 가지고 있다

=> components_array의 0번째 row, 10번째 col에 있는 값 : Topic #0에 대해서 피처 벡터화된 행렬에서 10번째 칼럼에 해당하는 피처가 Topic #0에 연관되는 수치 값 가지고 있다

* 함수 만들어 각 토픽별로 연관도가 높은 순으로 Word 나열

In [ ]:
def display_topics(model, feature_names, no_top_words):
  for topic_index, topic in enumerate(model.components_):
    print('Topic #', topic_index)

    # components_array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array 인덱스를 반환.
    topic_word_indexes = topic.argsort()[::-1]
    top_indexes = topic_word_indexes[:no_top_words]

    # top_indexes대상인 인덱스별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
    feature_concat = ' '.join([feature_names[i] for i in top_indexes])
    print(feature_concat)

# CountVectorizer객체 내의 전체 word의 명칭을 get_feautres_names()를 통해 추출
feature_names = count_vect.get_feature_names_out()

# 토픽별 가장 연관도가 높은 word를 15개만 추출
display_topics(lda, feature_names, 15)

Topic # 0
year 10 game medical health team 12 20 disease cancer 1993 games years patients good
Topic # 1
don just like know people said think time ve didn right going say ll way
Topic # 2
image file jpeg program gif images output format files color entry 00 use bit 03
Topic # 3
like know don think use does just good time book read information people used post
Topic # 4
armenian israel armenians jews turkish people israeli jewish government war dos dos turkey arab armenia 000
Topic # 5
edu com available graphics ftp data pub motif mail widget software mit information version sun
Topic # 6
god people jesus church believe christ does christian say think christians bible faith sin life
Topic # 7
use dos thanks windows using window does display help like problem server need know run


=> Topic #0의 경우 명확하지 않고 일반적인 단어, #1의 경우 컴퓨터 그래픽스 영역, #2는 기독교, #3는 의학, #4는 윈도우 운영체제, #5는 일반적인 단어, #6은 중동 분쟁 등, #7은 애매하지만 윈도우 운영체제 관련 일부 추출 / 특히 모토사이클, 야구 주제의 경우 명확한 주제어 추출x

## 08. 문서 유사도



---



### 문서 유사도 측정 방법 - 코사인 유사도

문서와 문서 간의 유사도 비교 -> **코사인 유사도** 사용

: 두 벡터 사이의 사잇각을 구해서 벡터의 상호 방향성이 얼마나 유사한지 수치로 적용한 것

### 두 벡터 사잇각

유사도 cos : 두 벡터의 내적을 총 벡터 크기의 합으로 나눈 것

* 코사인 유사도가 문서의 유사도 비교에 가장 많이 사용되는 이유

1. 무서를 피처 벡터화 변환하면 차원이 매우 많은 희소 행렬이 되기 쉽다 -> 희소 행렬 기반에서 문서와 문서 벡터 간의 크기에 기반한 유사도 지표는 정확도 떨어지기 쉽다

2. 문서가 매우 긴 경우 단어의 빈도수도 더 많을 것이기 때문에 이러한 빈도수에만 기반해서는 공정한 비교할 수 없다

In [ ]:
# cos_similarity() 함수 작성
import numpy as np

def cos_similarity(v1, v2):
  dot_product = np.dot(v1, v2)
  l2_norm = (np.sqrt(sum(np.square(v1))) * np.sqrt(sum(np.square(v2))))
  similarity = dot_product / l2_norm

  return similarity

In [ ]:
# 이 문서를 TF-IDF로 벡터화된 행렬로 변환
from sklearn.feature_extraction.text import TfidfVectorizer

doc_list = ['if you take the blue pill, the story ends',
            'if you take the red pill, you stay in Wonderland',
            'if you take the red pill, I show you how deep the rabbit hole goes']

tfidf_vect_simple = TfidfVectorizer()
feature_vect_simple = tfidf_vect_simple.fit_transform(doc_list)
print(feature_vect_simple.shape)

(3, 18)


In [ ]:
# TfidfVectorizer로 transform()한 결과는 희소 행렬이므로 밀집 행렬로 변환.
feature_vect_dense = feature_vect_simple.todense()

# 첫 번째 문장과 두 번째 문장의 피처 벡터 추출
vect1 = np.array(feature_vect_dense[0]).reshape(-1,)
vect2 = np.array(feature_vect_dense[1]).reshape(-1,)

# 첫 번째 문장과 두 번째 문장의 피처 벡터로 두 개 문장의 코사인 유사도 추출
similarity_simple = cos_similarity(vect1, vect2)
print('문장 1, 문장 2 Cosine 유사도: {0:.3f}'.format(similarity_simple))

문장 1, 문장 2 Cosine 유사도: 0.402


In [ ]:
# 첫 번째 문장과 세 번째 문장의 유사도도 측정
vect1 = np.array(feature_vect_dense[0]).reshape(-1,)
vect3 = np.array(feature_vect_dense[2]).reshape(-1,)
similarity_simple = cos_similarity(vect1, vect3)
print('문장 1, 문장 3 Cosine 유사도: {0:.3f}'.format(similarity_simple))

# 두 번째 문장과 세 번째 문장의 유사도도 측정
vect2 = np.array(feature_vect_dense[1]).reshape(-1,)
vect3 = np.array(feature_vect_dense[2]).reshape(-1,)
similarity_simple = cos_similarity(vect2, vect3)
print('문장 2, 문장 3 Cosine 유사도: {0:.3f}'.format(similarity_simple))

문장 1, 문장 3 Cosine 유사도: 0.404
문장 2, 문장 3 Cosine 유사도: 0.456


* 사이킷런은 코사인 유사도를 측정하기 위해 **sklearn.metrics.pairwise.cosine_similarity** API 제공
  + 두 개의 입력 파라미터 : 비교 기준이 되는 문서의 피처 행렬, 비교되는 문서의 피처 행렬
  + 희소 행렬, 밀집 행렬 모두 가능, 행렬 또는 배열 모두 가능
  + 별도의 변환 작업이 필요없다

In [ ]:
# 첫 번째 문서와 비교해 자신 문서인 첫 번째 문서, 그리고 두 번째, 세 번째 문서의 유사도 측정
from sklearn.metrics.pairwise import cosine_similarity

similarity_simple_pair = cosine_similarity(feature_vect_simple[0], feature_vect_simple)
print(similarity_simple_pair)

[[1.         0.40207758 0.40425045]]


=> 1이라는 값이 거슬린다면 feature_vect[1:] 이용해 비교 기준 문서 제외

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_simple_pair = cosine_similarity(feature_vect_simple[0], feature_vect_simple[1:])
print(similarity_simple_pair)

[[0.40207758 0.40425045]]


* 쌍으로(pair) 코사인 유사도 값 제공
  + 1번째 문서와 2,3번쨰 문서의 코사인 유사도, 2번째 문서와 1,3번째 문서의 코사인 유사도, 3번째 문서와 1,2번째 문서의 코사인 유사도를 ndarray 형태로 제공

In [ ]:
similarity_simple_pair = cosine_similarity(feature_vect_simple, feature_vect_simple)
print(similarity_simple_pair)
print('shape:', similarity_simple_pair.shape)

[[1.         0.40207758 0.40425045]
 [0.40207758 1.         0.45647296]
 [0.40425045 0.45647296 1.        ]]
shape: (3, 3)


### Opinion Review 데이터 세트를 이용한 문서 유사도 측정

* 문서 군집화 적용

In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

# 단어 원형 추출 함수
lemmar = WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

# 특수 문자 사전 생성: {33: None ...}
# ord(): 아스키 코드 생성
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

# 특수 문자 제거 및 단어 원형 추출
def LemNormalize(text):
    # 텍스트 소문자 변경 후 특수 문자 제거
    text_new = text.lower().translate(remove_punct_dict)

    # 단어 토큰화
    word_tokens = nltk.word_tokenize(text_new)

    # 단어 원형 추출
    return LemTokens(word_tokens)

In [ ]:
import pandas as pd
import glob, os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

nltk.download('punkt')
nltk.download('wordnet')

# 경로 이동
%cd /content/drive/MyDrive/data/topics
all_files=glob.glob(os.path.join('*.data'))
filename_list=[]
opinion_text=[]

for file_ in all_files:
  df=pd.read_table(file_, index_col=None, header=0, encoding='latin1')
  filename_=file_.split('\\')[-1]
  filename=filename_.split('.')[0]
  filename_list.append(filename)
  opinion_text.append(df.to_string())

document_df=pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})

tfidf_vect=TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',
                           ngram_range=(1,2), min_df=0.05, max_df=0.85)
feature_vect=tfidf_vect.fit_transform(document_df['opinion_text'])
km_cluster=KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label=km_cluster.labels_
cluster_centers=km_cluster.cluster_centers_
document_df['cluster_label']=cluster_label

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[Errno 2] No such file or directory: '/content/drive/MyDrive/data/topics'
/content


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


ValueError: ignored

* 호텔을 주제로 군집화된 문서 이용해 특정 문서와 다른 문서 간의 유사도 알아보기
  + 문서를 피처 벡터화해 변환하면 문서 내 단어에 출현 빈도와 같은 값을 부여해 각 문서가 단어 피처의 값으로 벡터화돼 -> cosine_similarity() 이용해 상호 비교해 유사도 확인

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# cluster_label=1인 데이터는 호텔로 군집화된 데이터임. DataFrame에서 해당 인덱스를 추출
hotel_indexes = document_df[document_df['cluster_label']==1].index
print('호텔로 군집화 된 문서들의 DataFrame Index:', hotel_indexes)

# 호텔로 군집화된 데이터 중 첫 번째 문서를 추출해 파일명 표시.
comparison_docname = document_df.iloc[hotel_indexes[0]]['filename']
print('##### 비교 기준 문서명 ', comparison_docname, ' 와 타 문서 유사도######')

''' document_df에서 추출한 Index 객체를 feature_vect로 입력해 호텔 군집화된 feature_vect 추출
이를 이용해 호텔로 군집화된 문서 중 첫 번째 문서와 다른 문서 간의 코사인 유사도 측정.'''
similarity_pair = cosine_similarity(feature_vect[hotel_indexes[0]], feature_vect[hotel_indexes])
print(similarity_pair)

* 첫 번째 문서와 다른 문서 간에 유사도가 높은 순으로 정렬하고 시각화
  + cosine_similarity()는 쌍 형태의 ndarray 반환 -> 판다스 인덱스로 이용하기 위해 reshape(-1)로 차원 변경

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 첫 번째 문서와 타 문서 간 유사도가 큰 순으로 정렬한 인덱스를 추출하되 자기 자신은 제외.
sorted_index = similarity_pair.argsort()[:, ::-1]
sorted_index = sorted_index[:, 1:]

# 유사도가 큰 순으로 hotel_indexes를 추출해 재정렬
hotel_sorted_indexes = hotel_indexes[sorted_index.reshape(-1)]

# 유사도가 큰 순으로 유사도 값을 재정렬하되 자기 자신은 제외
hotel_1_sim_value = np.sort(similarity_pair.reshape(-1))[::-1]
hotel_1_sim_value = hotel_1_sim_value[1:]

# 유사도가 큰 순으로 정렬된 인덱스와 유사도 값을 이용해 파일명과 유사도 값을 막대 그래프로 시각화
hotel_1_sim_df = pd.DataFrame()
hotel_1_sim_df['filename'] = document_df.iloc[hotel_sorted_indexes]['filename']
hotel_1_sim_df['similarity'] = hotel_1_sim_value

sns.barplot(x='similarity', y='filename', data=hotel_1_sim_df)
plt.title(comparison_docname)

=> 첫 번째 문서인 샌프란시스코의 베스트 웨스턴 호텔 화장실 리뷰(Best Western Hotel Bathroom Review)인 bathroom_bestwestern_hotel_sfo와 가장 비슷한 문서는 room_holidaty_inn_london -> 약 0.572의 코사인 유사도 값

## 09. 한글 텍스트 처리 - 네이버 영화 평점 감성 분석



---



### 한글 NLP 처리의 어려움

일반적으로 한글 언어 처리는 영어 등의 라틴어 처리보다 어렵다

주된 원인 - '띄어쓰기'와 '다양한 조사'

1. 한글은 띄어쓰기를 잘못하면 의미가 왜곡되어 전달
2. 조사는 주어나 목적어를 위해 추가. 워낙 경우의 수가 많아 -> 전처리 시 제거하기가 까다로워

### KoNLPy 소개

**KoNLPy** : 파이썬의 대표적인 한글 형태소 패키지

* 형태소 : 단어로서 의미를 가지는 최소 단위
  + 형태소 분석 : 말뭉치를 이러한 형태소 어근 단위로 쪼개고 각 형태소에 품사 태깅을 부착하는 작업

* KoNLPy : 기존의 C/C++, Java로 잘 만들어진 한글 형태소 엔진을 파이썬 래퍼(Wrapper) 기반으로 재작성한 패키지
  + 꼬꼬마(Kkma), 한나눔(Hannanum), Komoran, 은전한닢 프로젝트(Mecab), Twitter와 같이 5개의 형태소 분석 모듈 모두 사용 가능
  + 뛰어난 형태소 분석으로 인정받고 있는 Mecab의 경우 윈도우 환경에서는 구동x.

In [ ]:
!pip install --upgrade pip
!pip install JPype1-1.4.0-cp310-cp310-win_amd64.whl

In [ ]:
pip install konlpy

### 데이터 로딩

In [ ]:
import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/data/nsmc/ratings_train.txt', sep='\t')
train_df.head(3)

* 학습 데이터 세트의 0과 1의 Label 값 비율
  + 1이 긍정, 0이 부정 감성

In [ ]:
train_df['label'].value_counts()

=> 0과 1의 비율이 어느 한쪽으로 치우치지 않고 균등한 분포 나타냄

* train_df의 경우 리뷰 텍스트를 가지는 'documnet' 칼럼에 Null 일부 존재해 공백으로 변환
* 문자가 아닌 숫자의 경우 단어적인 의미로 부족해 파이썬 정규 표현식 모듈인 re 이용해 공백으로 변환
* 테스트 데이터 세트의 경우도 동일한 데이터 가공 수행

In [ ]:
import re

train_df = train_df.fillna(' ')
# 정규 표현식을 이용해 숫자를 공백으로 변경(정규 표현식으로 \d는 숫자를 의미함.)
train_df['document'] = train_df['document'].apply(lambda x : re.sub(r"\d+", " ", x))

# 테스트 데이터 세트를 로딩하고 동일하게 Null 및 숫자를 공백으로 변환
test_df = pd.read_csv('/content/drive/MyDrive/data/nsmc/ratings_test.txt', sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply(lambda x : re.sub(r"\d+", " ", x))

# id 칼럼 삭제 수행
train_df.drop('id', axis=1, inplace=True)
test_df.drop('id', axis=1, inplace=True)

* 각 문장을 한글 형태소 분석을 통해 형태소 단어로 토큰화
* 한글 형태소 엔진은 SNS 분석에 적합한 **Twitter 클래스** 이용
  + Twitter 객체의 **morphs()** 메서드 이용 : 입력 인자로 들어온 문장을 형태소 단어 형태로 토큰화해 list 객체로 반환
* 문장을 형태소 단어 형태로 반환하는 별도의 tokenizer 함수 생성

In [ ]:
from konlpy.tag import Twitter

# "Twitter" has changed to "Okt"
twitter = Twitter()
def tw_tokenizer(text):
  # 입력 인자로 들어온 텍스트를 형태소 단어로 토큰화해 리스트 형태로 반환
  tokens_ko = twitter.morphs(text)
  return tokens_ko

* 사이킷런의 TfidfVectorizer 이용해 TF-IDF 피처 모델 생성

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Twitter 객체의 morphs() 객체를 이용한 tokenizer를 사용. ngram_range는 (1,2)
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

* 로지스틱 회귀 이용해 분류 기반의 감성 분석 수행

In [ ]:
# 로지스틱 회귀를 이용해 감성 분석 분류 수행.
lg_clf = LogisticRegression(random_state=0)

# 파라미터 C 최적화를 위해 GridSearchCV를 이용.
params = {'C' : [1, 3.5, 4.5, 5.5, 10]}
grid_cv = GridSearchCV(lg_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfidf_matrix_train, train_df['label'])
print(grid_cv.best_params_, round(grid_cv.best_score_, 4))

* 테스트 세트 이용해 최종 감성 분석 예측 수행
  + 테스트 세트 이용해 예측할 때는 **학습할 때 적용한 TfidfVectorizer 그대로 사용**해야한다
  + 그래야만 학습 시 설정된 TfidfVectorizer의 피처 개수와 테스트 데이터를 TfidfVectorizer로 변환할 피처 개수가 같아진다

In [ ]:
from sklearn.metrics import accuracy_score

# 학습 데이터를 적용한 TfidfVectorizer를 이용해 테스트 데이터를 TF-IDF 값으로 피처 변환함.
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

# classifier는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도: ', accuracy_score(test_df['label'], preds))